In [36]:
import ast
import json
import sys
import math
import enum
import os
from copy import deepcopy
import re

In [37]:
class Primitives_Estimator:
    def __init__(self):
        pass
    def estimate_primitive_size(self, value):
        '''
        Estimate the size of a primitive value in bytes.
        '''
        return sys.getsizeof(value)
    def estimate_list_size(self,length):
        '''
        Estimate the size of a list in bytes based on its length.
        the size is calculated based on the number of elements and a precomputed capacity.
        size = size of #pointers ceiled
        '''
        def get_precomputed_capacity(length):
            if length == 0:
                return 0
            elif length <= 4:
                return 4
            elif length <= 8:
                return 8
            elif length <= 16:
                return 16
            elif length <= 25:
                return 25
            elif length <= 35:
                return 35
            elif length <= 49:
                return 49
            elif length <= 64:
                return 64
            else:
                return int(length * 1.025)
        base_size = sys.getsizeof([]) + 8 * get_precomputed_capacity(length)
        return base_size
    

In [38]:
class VariableToConstantTransformer(ast.NodeTransformer):
    def visit_Name(self, node):
        return ast.Constant(value=f"${node.id}")
class ConstantListToNamesTransformer(ast.NodeTransformer):
    def visit_Constant(self, node):
        # Only process string constants like "$var"
        if isinstance(node.value, str) and node.value.startswith("$"):
            return ast.Name(id=node.value[1:], ctx=ast.Load())

        # Handle Constant that wraps a list of strings like ["$x", "$y"]
        if isinstance(node.value, list):
            # Convert list elements recursively if they start with $
            elements = []
            for item in node.value:
                if isinstance(item, str) and item.startswith("$"):
                    elements.append(ast.Name(id=item[1:], ctx=ast.Load()))
                else:
                    elements.append(ast.Constant(value=item))
            return ast.List(elts=elements, ctx=ast.Load())

        return node

class AugAssignToExtend(ast.NodeTransformer):
    def visit_AugAssign(self, node):
        if isinstance(node.op, ast.Add):
            return ast.Expr(
                value=ast.Call(
                    func=ast.Attribute(
                        value=ast.copy_location(
                            ast.Name(id=node.target.id, ctx=ast.Load()),
                            node.target
                        ),
                        attr='extend',
                        ctx=ast.Load()
                    ),
                    args=[node.value],
                    keywords=[]
                )
            )
        return node
def convert_augassign_to_assign(node):
    if isinstance(node, ast.AugAssign):
        return ast.Assign(
            targets=[node.target],
            value=ast.BinOp(
                left=ast.copy_location(ast.Name(id=node.target.id, ctx=ast.Load()), node),
                op=node.op,
                right=node.value
            )
        )
    return node  # return unchanged if not AugAssign

# Example: wrap into a transformer
class AugAssignToAssignTransformer(ast.NodeTransformer):
    def visit_AugAssign(self, node):
        return convert_augassign_to_assign(node)
class InsertToAppend(ast.NodeTransformer):
    def visit_Call(self, node):
        self.generic_visit(node)  # Visit children nodes first
        if (
            isinstance(node.func, ast.Attribute) and
            node.func.attr == 'insert' and
            len(node.args) == 2
        ):
            # Change method name to 'append'
            node.func.attr = 'append'
            # Keep only the second argument (the value to insert)
            node.args = [node.args[1]]
        return node
def conv_len_assignment(tree):
    #! for x = len(var) case
    code = ast.unparse(tree)
    pattern = r"len\((\w+)\)"
    match = re.search(pattern, code)
    if match:
        var_name = match.group(1)
        if var_name in local_parser.vars:
            length = local_parser.vars[var_name][0] if local_parser.vars[var_name][2] == 'list' else 500
            modified_code = re.sub(pattern, str(length), code)
            print(f"Modified code: {modified_code}")  # Debugging: print the modified code
            tree = ast.parse(modified_code).body[0]  
            return tree
        else:
            raise ValueError(f"Variable {var_name} not found in local parser variables.")  
    #! for x = len(var[i])
    pattern = r"len\((\w+\[\d+\])\)" 
    match = re.search(pattern, code)
    if match:
        full_index_expr = match.group(1)  # e.g., numeric_data[0]
        var_name = full_index_expr.split('[')[0]  # Extract base variable name, e.g., numeric_data
        if var_name in local_parser.vars:
            length = str(local_parser.vars[var_name][0] if local_parser.vars[var_name][2] == 'list' else 500)  # Convert to string
            # Replace only the matched part
            modified_code = code[:match.start()] + str(length) + code[match.end():]
            print(f"Modified code: {modified_code}")  # Debugging: print the modified code
            tree = ast.parse(modified_code).body[0]
            return tree
        else:
            raise ValueError(f"Variable {var_name} not found in local parser variables.")
    return tree  # Return the original tree if no match is found

In [39]:
class Memory_Parser:
    transformer = VariableToConstantTransformer()
    transformer2 = ConstantListToNamesTransformer()
    transformer3 = AugAssignToExtend()
    transformer4 = InsertToAppend()
    class AssignTypes(enum.Enum):
        PRIMITIVE = "primitive"
        LIST = "list" 
    def __init__(self):
        self.primitives_estimator = Primitives_Estimator()
        self.vars = {'z' : (5, sys.getsizeof(0), 'int'),'y' :(10,1000,'list'), 'l' :(0,56,'list')}  #! varibles parsed so far (name: (value, memory, type))
        self.funcs = {'int':('int',0), 'str':('str',0), 'float':('float',0), 'bool':('bool',0), 'bytes':('bytes',0), 'bytearray':('bytearray',0), 'complex':('complex',0)
                      ,'list':('list',0)}  #! functions parsed so far (name: (type, memory))
        self.primitives=['int','str','float','bool','bytes','bytearray','complex','unk']  #! primitive types  
    def _reset(self):
        '''
        resets the memory parser.
        '''
        self.vars = {'z' : (5, sys.getsizeof(0), 'int'),'y' :(10,1000,'list'), 'l' :(0,56,'list')}
    def _hande_primitives_type_conversions(self, node):
        if isinstance(node.func, ast.Name) and node.func.id == 'int':  
                arg_val = self._evaluate_primtive_expression(node.args[0])
                return int(arg_val)
        elif isinstance(node.func, ast.Name) and node.func.id == 'str':
            return str(self._evaluate_primtive_expression(node.args[0]))
        elif isinstance(node.func, ast.Name) and node.func.id == 'float':
            return float(self._evaluate_primtive_expression(node.args[0]))
        return None
    def _handle_mathematical_ops(self, node, left_val, right_val):
        if isinstance(node.op, ast.Add):
            return left_val + right_val
        elif isinstance(node.op, ast.Sub):
            return left_val - right_val
        elif isinstance(node.op, ast.Mult):
            return left_val * right_val
        elif isinstance(node.op, ast.Div):
            return left_val / right_val
        elif isinstance(node.op, ast.FloorDiv):
            return left_val // right_val
        elif isinstance(node.op, ast.Pow):
            return left_val ** right_val
        elif isinstance(node.op, ast.Mod):
            return left_val % right_val
        else:
            raise ValueError(f"Unsupported operation: {type(node.op).__name__}")
        
    def _evaluate_primtive_expression(self, node):
        if isinstance(node, ast.Constant):  
            return node.value
        elif isinstance(node, ast.BinOp):  
            left_val = self._evaluate_primtive_expression(node.left)
            right_val = self. _evaluate_primtive_expression(node.right)
            result = self._handle_mathematical_ops(node, left_val, right_val)
            if result is not None:
                return result    
        elif isinstance(node, ast.Name):  
                if node.id in self.vars:
                    return self.vars[node.id][0]  
                else:
                    raise NameError(f"Variable '{node.id}' is not defined.")
        elif isinstance(node, ast.Call): 
            result = self._hande_primitives_type_conversions(node)
            if result is not None:
                return result
        else:
            raise TypeError(f"Unsupported AST node: {type(node).__name__}")   
           
    def _evaluate_primitive_assignment(self,stmt):
        '''
        
        evaluate primitive assignment statements.
        
        '''
        if isinstance(stmt.targets[0],ast.Subscript):  #! handle x[0] = 5
            return
        var_name = stmt.targets[0].id  
        result = self._evaluate_primtive_expression(stmt.value)
        memory = self.primitives_estimator.estimate_primitive_size(result)
        self.vars[var_name] = (result, memory, type(result).__name__) 
    def _assignment_type(self, node):
        '''
        
        Determine the type of assignment based on the AST node.
        supported types are primitive and list, primitive functions.
        
        '''
        if isinstance(node, ast.Constant):  
            if type(node.value).__name__ in self.primitives: 
                return self.AssignTypes.PRIMITIVE
            else:
                return self.AssignTypes.LIST
        elif isinstance(node, ast.List) or isinstance(node, ast.ListComp):
            return self.AssignTypes.LIST
        elif isinstance(node, ast.Name):
            if self.vars[node.id][2] in self.primitives:
                return self.AssignTypes.PRIMITIVE
            elif self.vars[node.id][2] == 'list':
                return self.AssignTypes.LIST
        elif isinstance(node, ast.Subscript):
            return self.AssignTypes.LIST  # Subscript is typically used for list indexing
        elif isinstance(node, ast.Call):
            if self.funcs[node.func.id][0] in self.primitives:
                return self.AssignTypes.PRIMITIVE
            elif self.funcs[node.func.id][0] == 'list':
                return self.AssignTypes.LIST 
        elif isinstance(node, ast.BinOp):
            left_type = self._assignment_type(node.left)
            right_type = self._assignment_type(node.right)
            if left_type == self.AssignTypes.LIST or right_type == self.AssignTypes.LIST:
                return self.AssignTypes.LIST
            elif left_type == self.AssignTypes.PRIMITIVE and right_type == self.AssignTypes.PRIMITIVE:
                return self.AssignTypes.PRIMITIVE
    def _evaluate_list_assignment(self, stmt,first=True):
        '''
        
        gets size of a list assignment statement.
        
        '''
        def handle_subscript(stmt):
                target = stmt.value.id
                if target not in self.vars:
                    raise NameError(f"Variable '{target}' is not defined syntax error.")
                if self.vars[target][2] != 'list':
                    raise TypeError(f"Variable '{target}' is not a list syntax error.")
                total_size = self.vars[target][1]
                total_length = self.vars[target][0]
                slice = stmt.slice
                lower = None
                upper = None
                step = 1
                size = None
                length = None
                if isinstance(slice, ast.Slice):
                    lower = slice.lower.value if isinstance(slice.lower, ast.Constant) else None
                    upper = slice.upper.value if isinstance(slice.upper, ast.Constant) else None
                    step = slice.step.value if isinstance(slice.step, ast.Constant) else 1
                elif isinstance(slice, ast.Constant):
                    lower = slice.value
                    upper = None
                    step = 1
                length = 0
                size = 0
                if lower is not None and upper is not None:
                    length = (upper - lower) // step
                    size = total_size//total_length * length + sys.getsizeof([])
                elif lower is not None and  upper is None:
                    if isinstance(slice, ast.Slice):
                        length = (total_length - lower)// step
                        size = (total_size // total_length) * length + sys.getsizeof([])
                    else:
                        size = total_size//total_length 
                        length =1
                        #! Note: this is an over estimation as it takes the size of the pointer into account 
                elif lower is None and upper is not None:
                    length = upper// step
                    size = total_size//total_length * length + sys.getsizeof([])
                return size, length
        def handle_BinOp(stmt):
            op = None
            left = None
            right = None
            if isinstance(stmt, ast.BinOp):
                op = stmt.op
                left = stmt.left
                right = stmt.right
            else:
                op = stmt.value.op
                left = stmt.value.left
                right = stmt.value.right

            left_size,left_length = _parse_list_elements_sizes(left)
            right_size,right_length = _parse_list_elements_sizes(right)
            total_size = left_size + right_size
            total_length = left_length if left_length  is not None else 0
            total_length += right_length if right_length is not None else 0
            total_length = 1 if total_length <= 0 else total_length
            if isinstance(op, ast.Add):
                if isinstance(left, ast.Name) and left.id in self.vars:
                    if self.vars[left.id][2] != 'list':
                        total_length = -1
                        total_size = total_size // total_length   if total_length > 0 else 0
                    else:
                        total_length += self.vars[left.id][0]
                        total_size+=self.primitives_estimator.estimate_list_size(self.vars[left.id][0]) - sys.getsizeof([]) 
                elif isinstance(left, ast.List):
                    total_length += len(left.elts)
                if isinstance(right, ast.Name) and right.id in self.vars:
                    if self.vars[right.id][2] != 'list':
                        total_size = total_size // total_length if total_length > 0 else 0
                        total_length = -1 
                    else:
                        total_length += self.vars[right.id][0]
                        total_size+=self.primitives_estimator.estimate_list_size(self.vars[right.id][0]) 
                elif isinstance(right, ast.List):
                    total_length += len(right.elts)
                return total_size, total_length
            else:
                total_size = total_size // total_length if total_length > 0 else 0
                total_length = -1
                return total_size, total_length
                
                
      
            
        def _parse_list_elements_sizes(node):
            if isinstance(node, ast.List):  
                sizes = [_parse_list_elements_sizes(el)[0] for el in node.elts]
                total_size = sum([size for size in sizes])
                total_length = len(node.elts)
                return total_size + self.primitives_estimator.estimate_list_size(total_length),0                     
            elif isinstance(node,ast.Constant):
                return sys.getsizeof(node.value),0
            elif isinstance(node, ast.Name):
                if node.id in self.vars:
                    return self.vars[node.id][1],0            
            elif (isinstance(node, ast.Subscript)):
                size, length = handle_subscript(node)
                return size, length
            elif isinstance(node, ast.BinOp):
                size, length = handle_BinOp(node)
                return size, length
            
        if isinstance(stmt.targets[0], ast.Subscript):
           return   
        var = stmt.targets[0].id
        multiplier = 1
        if (isinstance(stmt.value, ast.Call)):
            stmt=stmt.value.args[0]
        elif (isinstance(stmt.value, ast.Subscript)):
            size, length = handle_subscript(stmt.value)
            self.vars[var] = (length, size, 'list')
            return  
                        
        elif (isinstance(stmt.value, ast.BinOp)): 
            #! handle list multiplication
            op = stmt.value.op
            left = stmt.value.left
            right = stmt.value.right
            if isinstance(op, ast.Mult):
                list_val = None
                if isinstance(left,ast.Name) and left.id in self.vars and self.vars[left.id][2] == 'list':
                    multiplier = right.value if isinstance(right, ast.Constant) else self.vars[right.id][0]
                    length = self.vars[left.id][0]
                    list_val = ast.List(
                        elts=[ast.Constant(value=111111111) for _ in range(length)],
                        ctx=ast.Load()
                    )  
                elif isinstance(right,ast.Name) and right.id in self.vars and self.vars[right.id][2] == 'list':
                    multiplier = left.value if isinstance(right, ast.Constant) else self.vars[left.id][0]
                    list_val = ast.List(
                        elts=[ast.Constant(value=111111111) for _ in range(length)],
                        ctx=ast.Load()
                    )  
                #! Inner 1d indexing is assumed to be a primitive not a list
                elif (isinstance(left, ast.Subscript) or isinstance(right,ast.Subscript)):
                    size, length = handle_BinOp(stmt)
                    self.vars[var] = (1111111111, size, 'unk')  
                    return
                else:
                    multiplier = right.value if  isinstance(left, ast.List) else left.value
                    list_val=left if isinstance(left, ast.List) else right
                stmt=list_val
            else:
                size, length = handle_BinOp(stmt)
                if (not isinstance(op, ast.Add)):
                    self.vars[var] = (1111111111, size, 'unk')
                else:    
                    if length == -1:
                        self.vars[var] = (111111111, size, 'unk')
                    else:
                        self.vars[var] = (length, size, 'list')               
                return
        elif (isinstance(stmt.value, ast.ListComp)):
            elt = stmt.value.elt
            new_assign = ast.Assign(
                targets=[ast.Name(id=var, ctx=ast.Store())],
                value=elt
            )
            multiplier = (
                stmt.value.generators[0].iter.args[0].value
                if isinstance(stmt.value.generators[0].iter, ast.Call)
                and isinstance(stmt.value.generators[0].iter.args[0], ast.Constant)
                else 1
            )            
            self._evaluate_list_assignment(new_assign)
            length = self.vars[var][0] * multiplier
            size = self.vars[var][1] * multiplier
            self.vars[var] = (length, size, 'list')
            return
        else:
            stmt=stmt.value
        list_length = len(stmt.elts)
        list_length = list_length * multiplier
        memory = self.primitives_estimator.estimate_list_size(list_length)
        elements_size = _parse_list_elements_sizes(stmt)[0] * multiplier
        if first:
           self.vars[var] = (list_length,elements_size,'list')
        else:
            self.vars[var] = (self.vars[var][0] + list_length, self.vars[var][1] + elements_size, 'list')
        
          
    def _assignmemt_handler(self,tree):
        stmt = tree
        if self._assignment_type(stmt.value) == self.AssignTypes.PRIMITIVE:
            self._evaluate_primitive_assignment(stmt)
        elif self._assignment_type(stmt.value) == self.AssignTypes.LIST :
            print("List Assignment")            
            self._evaluate_list_assignment(stmt)
    def _handle_list_insertion(self, var,func,args,in_loop = 1):
        if func == 'append':
            #! start with $ then a variable name
            if isinstance(args[0], str) and args[0].startswith("$"):
               args[0] = args[0][1:]  # Remove the leading '$'
               if args[0] not in self.vars:
                   raise NameError(f"Variable '{args[0]}' is not defined syntax error.")
               args[0] = self.vars[args[0]][0]  # Get the value of the variable
            for i in range(in_loop):
                new_node = ast.Assign(
                    targets=[ast.Name(id=var, ctx=ast.Store())],
                    value=ast.List(elts=[ast.Constant(value=args[0])], ctx=ast.Load())
                )
                new_node = self.transformer2.visit(new_node)
                # print(f"New Node: {ast.dump(new_node)}")
                self._evaluate_list_assignment(new_node, False)
        elif func == 'extend':  
            #! extend treats immutable objects as if they were re-created in memory together with their pointers.
            
            flattened_args = []
            for sublist in args:
                for item in sublist:
                    if isinstance(item, str) and item.startswith("$"):
                        item = item[1:]
                        if item not in self.vars:
                            raise NameError(f"Variable '{item}' is not defined syntax error.")
                        if self.vars[item][2] == 'list':
                            length = self.vars[item][0]
                            size = self.vars[item][1]
                            total_size = size + self.primitives_estimator.estimate_list_size(length) -  sys.getsizeof([]) #! should be 2 one for get the size of the list and one for the pointer in the original list
                            self.vars[var] = (self.vars[var][0] + length, self.vars[var][1] + total_size, 'list')
                            continue
                    flattened_args.append(item)
            args = flattened_args
            for i in range(in_loop):
                for arg in args:
                    new_node = ast.Assign(
                        targets=[ast.Name(id=var, ctx=ast.Store())],
                        value=ast.List(elts=[ast.Constant(value=arg)], ctx=ast.Load())
                    )
                    new_node = self.transformer2.visit(new_node)
                    # print(f"New Node: {ast.dump(new_node)}")
                    self._evaluate_list_assignment(new_node, False)
    def _insertion_handler(self, tree,in_loop = 1):
        def extract_call_info(tree):
            for node in ast.walk(tree):
                if isinstance(node, ast.Call):
                    func_node = node.func
                    if isinstance(func_node, ast.Attribute):
                        var_id = func_node.value.id  # e.g., 'x'
                        func_type = func_node.attr   # e.g., 'append'
                        args = []
                        for arg in node.args:
                            if isinstance(arg, ast.Name):
                                # Capture variable name as a string
                               args.append([f"${arg.id}"])
                            elif isinstance(arg, ast.Call):
                                args.append(111111111111111)
                            else:
                                node = self.transformer.visit(node)
                                try:
                                    args.append(ast.literal_eval(arg))
                                except (ValueError, SyntaxError):
                                    raise ValueError(f"Unsupported argument type: {type(arg).__name__}")
                        return var_id, func_type, args
         
        if isinstance(tree, ast.AugAssign): #! for handling +=
            tree = self.transformer3.visit(tree)
        else: #1 to replace insert with append
            contains_insert = any(
                isinstance(node, ast.Call) and
                isinstance(node.func, ast.Attribute) and
                node.func.attr == 'insert'
                for node in ast.walk(tree)
            )
            if contains_insert:
                tree = self.transformer4.visit(tree)
        var, func_type, args = extract_call_info(tree)
        if not var in self.vars:
            raise NameError(f"Variable '{var}' is not defined syntax error.")
        if self.vars[var][2] == 'list':
           self._handle_list_insertion(var, func_type,args, in_loop)   
            
        
    def _handle_list_deletion(self, var, func_type,in_loop = 1):
        if func_type == 'pop':
            if self.vars[var][0] == 0:
                raise IndexError(f"pop from empty list syntax error.")
            for i in range(in_loop):
                total_size = self.vars[var][1]
                total_length = self.vars[var][0]
                new_size = total_size - (total_size // total_length)  
                self.vars[var] = (total_length - 1, new_size, 'list')
            return
        elif func_type == 'clear':
            self.vars[var] = (0, sys.getsizeof([]), 'list')  # Reset to empty list
        elif func_type == 'remove':
            for i in range(in_loop):
                new_length = self.vars[var][0] - 1
                new_size = self.vars[var][1] - (self.vars[var][1] // self.vars[var][0])
                self.vars[var] = (new_length, new_size, 'list') 
    def _deletion_handler(self, tree, in_loop = 1):
        def extract_call_info(tree):
            for node in ast.walk(tree):
                if isinstance(node, ast.Call):
                    func_node = node.func
                    if isinstance(func_node, ast.Attribute):
                        var_id = func_node.value.id  # e.g., 'x'
                        func_type = func_node.attr   # e.g., 'pop'
                        return var_id, func_type
        var_id, func_type = None, None
        if  isinstance(tree, ast.Delete): #! handle del statements
            stmt = tree
            subscript = stmt.targets[0]
            var_id = subscript.value.id
            slice = subscript.slice
            lower = None
            upper = None
            step = 1
            if isinstance(slice, ast.Slice):
                lower = slice.lower.value if isinstance(slice.lower, ast.Constant) else None
                upper = slice.upper.value if isinstance(slice.upper, ast.Constant) else None
                step = slice.step.value if isinstance(slice.step, ast.Constant) else 1
            elif isinstance(slice, ast.Constant):
                lower = slice.value
                upper = None
                step = 1 
            total_length = self.vars[var_id][0]
            total_size = self.vars[var_id][1]
            if lower is not None and upper is not None:
                length = (upper - lower) // step
                size = total_size // total_length * length
                self.vars[var_id] = (total_length - length, total_size - size, 'list')
            elif lower is not None and upper is None:
                if isinstance(slice, ast.Slice):
                        length = (total_length - lower)// step
                        size = total_size - (total_size // total_length) * length
                        length = total_length - length 
                else:
                    length = total_length - 1
                    size = total_size - total_size // total_length
                self.vars[var_id] = (length, size, 'list')
            elif lower is None and upper is not None:
                length = upper// step
                size = total_size - total_size//total_length * length
                length = total_length - length
                self.vars[var_id] = (length, size, 'list')     

                
            return                
        else:
            var_id, func_type = extract_call_info(tree)
        if var_id not in self.vars:
            raise NameError(f"Variable '{var_id}' is not defined syntax error.")
        if self.vars[var_id][2] != 'list':
            raise TypeError(f"Variable '{var_id}' is not a list syntax error.")
        self._handle_list_deletion(var_id, func_type, in_loop)
                    
         
    def _list_method_handler(self, tree):
        def extract_call_info(tree):
            for node in ast.walk(tree):
                if isinstance(node, ast.Call):
                    func_node = node.func
                    if isinstance(func_node, ast.Attribute):
                        var_id = func_node.value.id  # e.g., 'x'
                        func_type = func_node.attr   # e.g., 'pop'
                        return var_id, func_type
        var, func_type = extract_call_info(tree)
        #! returns length,size
        if func_type == ['reverse','sort']:
            if var not in self.vars:
                raise NameError(f"Variable '{var}' is not defined syntax error.")
            if self.vars[var][2] != 'list':
                raise TypeError(f"Variable '{var}' is not a list syntax error.")
            # Reversing a list does not change its size or length
            return None,None
        elif func_type in ['count','index']:
            return 0, sys.getsizeof(10000000)  #! hardcoded maybe changed later (or maybe not :( )
        elif func_type in ['copy']:
            return self.vars[var][0],self.vars[var][1]
            
        
    
        # print(f"Variable ID: {var}, Function Type: {func_type}, Arguments: {args}")
    def _file_handler(self, tree):
        '''
        gets the file metadata and records it in the dictionary.
        '''
        def extract_file_info(tree):
            file_path = None
            list_name = None

            for node in ast.walk(tree):
                # Find open() call and extract file path
                if isinstance(node, ast.Call) and isinstance(node.func, ast.Name) and node.func.id == 'open':
                    if len(node.args) >= 1 and isinstance(node.args[0], ast.Constant):
                        file_path = node.args[0].value

                # Find assignment to variable from file.readlines()
                if isinstance(node, ast.Assign):
                    if isinstance(node.value, ast.Call) and isinstance(node.value.func, ast.Attribute):
                        if node.value.func.attr == 'readlines' and isinstance(node.targets[0], ast.Name):
                            list_name = node.targets[0].id

            return file_path, list_name
        file_path ,var = extract_file_info(tree)
        file_size = os.path.getsize(file_path) + sys.getsizeof([])  #! add the size of the list pointer
        length = 0
        with open(file_path, 'rb') as f:
            length = sum(1 for _ in f)
        self.vars[var] = (length, file_size, 'list')
    def _get_return_size_length(self,node):
        var_name = None
        lower = None
        upper = None
        step = None
        ret_stmt = node.body[0]
        if not isinstance(ret_stmt, ast.Return):
            return None

        value = ret_stmt.value
   
        if isinstance(value, ast.Name):
            var_name = value.id

        # Case: return x[0] or return x[0:4] or return x[0:8:2]
        if isinstance(value, ast.Subscript) and isinstance(value.value, ast.Name):
            var_name = value.value.id

            # Case: return x[0]
            if isinstance(value.slice, ast.Constant):
                lower = value.slice.value
                upper = lower + 1
                step = 1
    
            # Case: return x[0:4] or x[0:8:2]
            elif isinstance(value.slice, ast.Slice):
                def get_val(val): return val.value if isinstance(val, ast.Constant) else None
                lower = get_val(value.slice.lower)
                upper = get_val(value.slice.upper)
                step = get_val(value.slice.step) if get_val(value.slice.step) is not None else 1
            
        if var_name not in self.vars:
            raise NameError(f"Variable '{var_name}' is not defined syntax error.")
        original_length, original_size, _ = self.vars[var_name]
        if lower is None:
            length = self.vars[var_name][0] if self.vars[var_name][2] == 'list' else 1
            return original_size, length
        else:
            new_length = (upper - lower) // step
            new_size = int(original_size // original_length * new_length)
            return new_size, new_length
    def _handle_loop_footprint(self, node):
        def get_number_outer_iterations(node):
            loop_expr = None
            iter_node = node.iter
            #! no subscript within the loop
            if isinstance(iter_node, ast.Subscript):
                iter_node = ast.Name(id=iter_node.value.id, ctx=ast.Load())
            # Case 1: for i in x:
                
            if isinstance(iter_node, ast.Name):
                loop_expr = f"len({iter_node.id})"
            # Case 2: for i in range(...):
            elif isinstance(iter_node, ast.Call) and isinstance(iter_node.func, ast.Name):
                if iter_node.func.id == 'range':
                    if len(iter_node.args) == 1:
                        # range(N)
                        if isinstance(iter_node.args[0], ast.Constant):
                            loop_expr = str(iter_node.args[0].value)
                        else:
                            loop_expr = ast.unparse(iter_node.args[0])
                            if not loop_expr.startswith("len("):
                                if loop_expr in self.vars:
                                    loop_expr = self.vars[loop_expr][0]
                                else:
                                    raise ValueError(f"Variable '{loop_expr}' is not defined syntax error.") 
                    elif len(iter_node.args) in [2, 3]:
                        start = ast.unparse(iter_node.args[0])
                        stop = ast.unparse(iter_node.args[1])
                        if len(iter_node.args) == 3:
                            step = ast.unparse(iter_node.args[2])
                            loop_expr = f"({stop} - {start}) // {step}"
                        else:
                            loop_expr = f"{stop} - {start}"
                # Case 3: for index, name in enumerate(names):
                elif iter_node.func.id == 'enumerate' and len(iter_node.args) == 1:
                    arg_expr = ast.unparse(iter_node.args[0])
                    loop_expr = f"len({arg_expr})"
                # Case 4: for i, j in zip(a, b):
                elif iter_node.func.id == 'zip':
                    zipped_lens = []
                    for arg in iter_node.args:
                        zipped_lens.append(f"len({ast.unparse(arg)})")
                    loop_expr = f"min({', '.join(zipped_lens)})"
            # Try resolving len(x) from known vars
            if isinstance(loop_expr, int):
                return loop_expr 
            if loop_expr:
                if loop_expr.startswith("len(") and loop_expr.endswith(")"):
                    var = loop_expr[4:-1].strip()
                    if var in self.vars:
                        loop_expr = self.vars[var][0]
                return loop_expr
            else:
                raise ValueError("Unsupported loop iteration expression.")
        def has_inner_for_loop(node):
            if not isinstance(node, ast.For):
                return False

            for child in ast.walk(node):
                if child is not node and isinstance(child, ast.For):
                    return True
            return False
        def get_assignment_targets(assign_node):
            targets = []
            target = assign_node.target
            if isinstance(target, ast.Name):
                targets.append(target.id)
            elif isinstance(target, ast.Tuple):
                targets.extend([elt.id for elt in target.elts if isinstance(elt, ast.Name)])
            return targets
        def handle_nested_loops(node, depth=0,n_outer_iterations=None, n_inner_iterations=None):
            if not isinstance(node, ast.For):
                return
            print("  " * depth + f"Handling loop at depth {depth}")
            n_iter = n_outer_iterations if depth == 0 else n_inner_iterations
            for stmt in node.body:
                # print(ast.dump(stmt, indent=4))
                if isinstance(stmt, ast.For):
                    original_vars = self.vars.copy()
                    targets = get_assignment_targets(stmt)
                    costs = get_iterable_cost_expr_only(stmt, self.vars)
                    for i,target in enumerate(targets):
                        self.vars[target] = (1,costs[i], 'unk')
                    handle_nested_loops(stmt, depth + 1, n_outer_iterations=n_iter, n_inner_iterations=n_inner_iterations)
                    for key in list(self.vars.keys()):
                        if key not in original_vars:
                            del self.vars[key]
                elif isinstance(stmt, ast.If):
                    self._handle_if_footprint(stmt)
                else:
                    if isinstance(stmt, ast.Assign):
                        stmt = conv_len_assignment(deepcopy(stmt))
                        self._assignmemt_handler(stmt)
                    elif  isinstance(stmt, ast.AugAssign):
                        size_before_loop = self.vars[stmt.target.id][1]
                        len_before_loop = self.vars[stmt.target.id][0]
                        self._insertion_handler(deepcopy(stmt))
                        size_after_loop = self.vars[stmt.target.id][1]
                        len_after_loop = self.vars[stmt.target.id][0]
                        size_increment = size_after_loop - size_before_loop
                        len_increment = len_after_loop - len_before_loop
                        new_size = size_before_loop + size_increment * int(n_iter)
                        new_length = len_before_loop + len_increment * int(n_iter)
                        self.vars[stmt.target.id] = (new_length,new_size, 'list')
                    elif isinstance(stmt, ast.Expr):
                        func = stmt.value.func.attr 
                        if func in ['insert', 'append', 'extend']:
                            size_before_loop = self.vars[stmt.value.func.value.id][1]
                            len_before_loop = self.vars[stmt.value.func.value.id][0]
                            self._insertion_handler(deepcopy(stmt))
                            size_after_loop = self.vars[stmt.value.func.value.id][1]
                            len_after_loop = self.vars[stmt.value.func.value.id][0]
                            size_increment = size_after_loop - size_before_loop
                            len_increment = len_after_loop - len_before_loop
                            new_size = size_before_loop + size_increment * int(n_iter)
                            new_length = len_before_loop + len_increment * int(n_iter)
                            self.vars[stmt.value.func.value.id] = (new_length,new_size, 'list')
                        elif func in ['pop', 'remove','clear']:
                            size_before_loop = self.vars[stmt.value.func.value.id][1]
                            len_before_loop = self.vars[stmt.value.func.value.id][0]
                            self._deletion_handler(deepcopy(stmt))
                            size_after_loop = self.vars[stmt.value.func.value.id][1]
                            len_after_loop = self.vars[stmt.value.func.value.id][0]
                            size_decrement = size_after_loop - size_before_loop
                            len_decrement = len_after_loop - len_before_loop
                            new_size = size_before_loop + size_decrement * int(n_iter)
                            new_length = len_before_loop + len_decrement * int(n_iter)
                            self.vars[stmt.value.func.value.id] = (new_length,new_size, 'list')
                    elif  isinstance(stmt, ast.Delete):
                            for target in stmt.targets:
                                if isinstance(target, ast.Subscript) and isinstance(target.value, ast.Name):    
                                    var_name = target.value.id        
                                    size_before_loop = self.vars[var_name][1]
                                    len_before_loop = self.vars[var_name][0]
                                    self._deletion_handler(deepcopy(stmt))
                                    size_after_loop = self.vars[var_name][1]
                                    len_after_loop = self.vars[var_name][0]
                                    size_decrement = size_after_loop - size_before_loop
                                    len_decrement = len_after_loop - len_before_loop
                                    #! max is to avoid negative sizes or lengths
                                    new_size = max(size_before_loop + size_decrement * int(n_iter),0)
                                    new_length = max(len_before_loop + len_decrement * int(n_iter),0)
                                    self.vars[var_name] = (new_length,new_size, 'list')
            print("  " * (depth + 1) + f"Statement: {ast.dump(stmt)}")
            print(self.vars)
        def get_iterable_cost_expr_only(for_node, vars_dict):
            iter_expr = for_node.iter
            if isinstance(iter_expr, ast.Subscript):
                iter_expr = ast.Name(id=iter_expr.value.id, ctx=ast.Load())
            cost_exprs = []

            def var_expr(name):
                return self.vars[name][1]//self.vars[name][0]

            if isinstance(iter_expr, ast.Call):
                func_name = getattr(iter_expr.func, 'id', None)

                if func_name == 'enumerate':
                    arg = iter_expr.args[0]
                    if isinstance(arg, ast.Name):
                        name = arg.id
                        cost_exprs.append(sys.getsizeof(100000))
                        cost_exprs.append(var_expr(name))

                elif func_name == 'zip':
                    for arg in iter_expr.args:
                        if isinstance(arg, ast.Name):
                            cost_exprs.append(var_expr(arg.id))

                elif func_name == 'range':
                    cost_exprs.append(sys.getsizeof(100000))

            elif isinstance(iter_expr, ast.Name):
                cost_exprs.append(var_expr(iter_expr.id))

            elif isinstance(iter_expr, ast.Call) and isinstance(iter_expr.func, ast.Name) and iter_expr.func.id == 'len':
                cost_exprs.append("sys.getsizeof(100000)")

            elif isinstance(iter_expr, ast.Constant) and isinstance(iter_expr.value, int):
                cost_exprs.append("sys.getsizeof(100000)")
            return cost_exprs
        
        original_vars = self.vars.copy()
        n_outer_iterations = get_number_outer_iterations(node)
        n_inner_iterations = 500 if has_inner_for_loop(node) else None
        targets = get_assignment_targets(node)
        costs = get_iterable_cost_expr_only(node, self.vars) 
        for i,target in enumerate(targets):
            self.vars[target] = (1,costs[i], 'unk')  # Initialize targets to empty lists

        handle_nested_loops(node, depth=0, n_outer_iterations=n_outer_iterations, n_inner_iterations=n_inner_iterations)
        for key in list(self.vars.keys()):
            if key not in original_vars:
                del self.vars[key]
        print(self.vars)
    def _handle_if_footprint(self, node):
        def extract_all_if_blocks(root_if_node):
            blocks = []
            current = root_if_node
            while isinstance(current, ast.If):
                blocks.append(ast.Module(body=deepcopy(current.body), type_ignores=[]))
                if len(current.orelse) == 1 and isinstance(current.orelse[0], ast.If):
                    current = current.orelse[0]  # move to next `elif`
                else:
                    if current.orelse:
                        blocks.append(ast.Module(body=deepcopy(current.orelse), type_ignores=[]))  # final `else` block
                    break
            return blocks
        def handle_if_blocks(blocks):
            vars_footprints_dicts= []
            for block in blocks:
                original_vars = self.vars.copy()
                for stmt in block.body:
                    if isinstance(stmt, ast.Assign):
                        stmt = conv_len_assignment(deepcopy(stmt))
                        self._assignmemt_handler(stmt)
                    elif isinstance(stmt, ast.AugAssign):
                        self._insertion_handler(stmt)
                    elif isinstance(stmt, ast.Expr):
                        if isinstance(stmt.value, ast.Call) and isinstance(stmt.value.func, ast.Attribute):
                            func = stmt.value.func.attr
                            if func in ['insert', 'append', 'extend']:
                                self._insertion_handler(stmt)
                            elif func in ['pop', 'remove', 'clear']:
                                self._deletion_handler(stmt)
                    elif isinstance(stmt, ast.Delete):
                        for target in stmt.targets:
                            if isinstance(target, ast.Subscript) and isinstance(target.value, ast.Name):
                                self._deletion_handler(stmt)
                    elif isinstance(stmt, ast.If):
                        self._handle_if_footprint(stmt)
                    elif isinstance(stmt, ast.For):
                        self._handle_loop_footprint(stmt)
                vars_footprints_dicts.append(self.vars.copy())
                self.vars = original_vars.copy()  #! Reset to original vars after each block
            footprints = []
            # print(vars_footprints_dicts)
            for item in vars_footprints_dicts:
                total_size = sum(value[1]  for value in item.values())  #! sum the values of the dicts
                footprints.append(total_size)
            # print(footprints)
            max_index = footprints.index(max(footprints))  #! get the index of the max footprint
            # print(max_index)
            self.vars = vars_footprints_dicts[max_index]  #! set the vars to the max footprint dict
            for key in list(self.vars.keys()):
                if key not in original_vars:
                    del self.vars[key]
                
        # original_vars = self.vars.copy()
        blocks = extract_all_if_blocks(node)
        # for block in blocks:
        #     print(ast.dump(block, indent=4))
        handle_if_blocks(blocks)
       
            
       
        
        


In [40]:
parser = Memory_Parser()
print("Testcase 1")
x = 'x =  []'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Original size: {sys.getsizeof([])}")
print(f"Estimated size: {parser.vars['x'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 2")
x = 'x = [[[]]]'
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size: {parser.vars['x'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 3")
x = 'x = [1,2,3]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size: {parser.vars['x'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 4")
x= 'x = [[1,2,3],[1.0,2.0,"fff"]]'
tree = ast.parse(x)
parser._assignmemt_handler(ast.parse(tree.body[0]))
print(f"Estimated size: {parser.vars['x'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 5")
x = 'x = [1,2,3] * 10'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size: {parser.vars['x'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 6")
x = 'x = 10 * [1,2,3]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size: {parser.vars['x'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 7")
x = 'x = [z]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size: {parser.vars['x'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 8")
x = 'x=6'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size: {parser.vars['x'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 9")
x = 'x  = [[0] * 3 for _ in range(2)]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size: {parser.vars['x'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 10")
x = '''try:
    with open('test_file.txt', 'r') as file:
        lines = file.readlines()
except FileNotFoundError:
    print("File not found. Please ensure 'test_file.txt' exists in the current directory.")
print(lines)
'''
tree = ast.parse(x) 
parser._file_handler(tree)
print(f"Estimated size: {parser.vars['lines'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 11")
x ='l.append([4,5])'  #! append a list to a list
tree = ast.parse(x)
parser._insertion_handler(tree)
print(f"Estimated size after append: {parser.vars['l'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 12")
x = 'l.extend([[3],[4,5]])'
tree = ast.parse(x)
parser._insertion_handler(tree)
print(f"Estimated size after extend: {parser.vars['l'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 13")
x = 'l.extend([[y],[3]])'
tree = ast.parse(x)
parser._insertion_handler(tree)
print(f"Estimated size after extend: {parser.vars['l'][1]}")
parser._reset()  
print("----------------------------------------------------------------------------------")
print("Testcase 14")
x = 'l+=y'
tree = ast.parse(x)
parser._insertion_handler(tree.body[0])
print(f"Estimated size after +=: {parser.vars['l'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 15")
x = 'l+=[3,4,[5]]'
tree = ast.parse(x)
parser._insertion_handler(tree.body[0])
print(f"Estimated size after +=: {parser.vars['l'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 16")
x = 'l = [1,2,3] + [4,5]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['l'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 17")
x = 'l = [1,2,3] + [z]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['l'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 18")
x = 'l = y + [y]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['l'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 19")
x = 'l.insert(1,[z])'
tree = ast.parse(x)
parser._insertion_handler(tree)
print(f"Estimated size after assignment: {parser.vars['l'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 20")
x = 'l = y[1]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['l'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 21")
x = 'l = y[1:5]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['l'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 22")
x = 'l = y[1:5:2]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['l'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 23")
x = 'y.pop(1)'
tree = ast.parse(x)
parser._deletion_handler(tree)
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 24")
x = 'y.clear(1)'
tree = ast.parse(x)
parser._deletion_handler(tree)
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 25")
x = 'y.remove(1)'
tree = ast.parse(x)
parser._deletion_handler(tree)
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 26")
x = 'del y[1]'
tree = ast.parse(x)
parser._deletion_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 27")
x = 'del y[1:5]'
tree = ast.parse(x)
parser._deletion_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 28")
x = 'del y[1:5:2]'
tree = ast.parse(x)
parser._deletion_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 29")
x = 'y.copy()'
tree = ast.parse(x)
print(f"Estimated size after assignment: {parser._list_method_handler(tree)}")  #! should return the size of the list
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 30")
x = 'y.index(1,3,5)'
tree = ast.parse(x)
print(f"Estimated size after assignment: {parser._list_method_handler(tree)}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 31")
x = 'y.sort()'
tree = ast.parse(x)
print(f"Estimated size after assignment: {parser._list_method_handler(tree)}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 32")
x = 'y.reverse()'
tree = ast.parse(x)
print(f"Estimated size after assignment: {parser._list_method_handler(tree)}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 33")
x = 'y.reverse()'
tree = ast.parse(x)
print(f"Estimated size after assignment: {parser._list_method_handler(tree)}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 34")
x = 'return y'
tree = ast.parse(x)
print(f"Estimated size after assignment: {parser._get_return_size_length(tree)}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 35")
x = 'return y[0]'
tree = ast.parse(x)
print(f"Estimated size after assignment: {parser._get_return_size_length(tree)}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 36")
x = 'return y[0:4]'
tree = ast.parse(x)
print(f"Estimated size after assignment: {parser._get_return_size_length(tree)}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 36")
x = 'return y[0:8:3]'
tree = ast.parse(x)
print(f"Estimated size after assignment: {parser._get_return_size_length(tree)}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 37")
x = 'y =  y[0:]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 38")
x = 'y =  y[:8]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 39")
x = 'y =  y[:8:2]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 40")
x = 'y =  y[0:] + y[0:]+ y[5:8] + y'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 41")
x = '''for i in y:
        x = 5
        y += [x]
        y.append(y)'''
tree = ast.parse(x)
parser._handle_loop_footprint(tree.body[0])
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 42")
x = '''for i in range(len(y)):
        y += [5,5,5,5]
        y.pop()'''
tree = ast.parse(x)
parser._handle_loop_footprint(tree.body[0])
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 43")
x = '''for index, name  in enumerate(y):
        y += [5]
        y.append(y)
        del y[0:]'''
tree = ast.parse(x)
parser._handle_loop_footprint(tree.body[0])
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 44")
x = '''for i in range(1000):
        y += [5]
        y.append(y)'''
tree = ast.parse(x)
parser._handle_loop_footprint(tree.body[0])
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 45")
x = '''for row in y[1:]:
        new_row = []
        for value in row:
            new_value = int(value) + 1  
            new_row.append(new_value)'''
tree = ast.parse(x)
parser._handle_loop_footprint(tree.body[0])
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 46")
x = 'y =  y[0] + z'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 47")
x = 'y =  (2*z) + z'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 48")
x = 'y =  (y[0]+y[1:8:2])+ y[0:4]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 49")
x = 'y =  y[0] + 2 / 9'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 50")
x = 'y =  y[0] ** y[2] / 9'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 51")
x = 'y =  y * 2'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 52")
x = 'y =  y * 8'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 53")
x = 'y[i] = 5'
tree = ast.parse(x)
tree = AugAssignToAssignTransformer().visit(tree)
ast.fix_missing_locations(tree)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 54")
x = 'y[1:4:2] = [5,8,8,7]'
tree = ast.parse(x)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------")
print("Testcase 55")
x = 'y*= 5'
tree = ast.parse(x)
tree = AugAssignToAssignTransformer().visit(tree)
ast.fix_missing_locations(tree)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 56")
x = 'y*= 5'
tree = ast.parse(x)
tree = AugAssignToAssignTransformer().visit(tree)
ast.fix_missing_locations(tree)
parser._assignmemt_handler(tree.body[0])
print(f"Estimated size after assignment: {parser.vars['y'][1]}")
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 57")
x = '''if 10 > 5:
    x = 9
    y.append(x)
    if x > 5:
       y = 3
    elif x == 4:
       y = 5
    else: 
       y = 6
elif x < 3:
    y.append(8)
elif y == 5:
    y.append(9)
else:
    y.append(7)'''
tree = ast.parse(x)
tree = AugAssignToAssignTransformer().visit(tree)
ast.fix_missing_locations(tree)
parser._handle_if_footprint(tree.body[0])
print(parser.vars)
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 58")
x = '''if 10 > 5:
    t = 5
    if x < 3:
        y.extend(y)
        y*=2
        for row in y:
            y.append(row)
            if 8 < 7:
               c = 5
    elif x >= 3:
        y = 9
elif x < 3:
    y.extend(y)
else:
    y.append(7)'''
tree = ast.parse(x)
tree = AugAssignToAssignTransformer().visit(tree)
ast.fix_missing_locations(tree)
parser._handle_if_footprint(tree.body[0])
print(parser.vars)
parser._reset()
print("----------------------------------------------------------------------------------")
print("Testcase 58")
x = '''for i in range(z):
        y += [5,5,5,5]
        y.pop()'''
tree = ast.parse(x)
parser._handle_loop_footprint(tree.body[0])
parser._reset()
print("----------------------------------------------------------------------------------")
x = 'y.append(float(z))'
tree = ast.parse(x)
parser._insertion_handler(tree.body[0])
parser._reset()
print("----------------------------------------------------------------------------------")

Testcase 1
List Assignment
Original size: 56
Estimated size: 56
----------------------------------------------------------------------------------
Testcase 2
List Assignment
Estimated size: 56
----------------------------------------------------------------------------------
Testcase 3
List Assignment
Estimated size: 172
----------------------------------------------------------------------------------
Testcase 4
List Assignment
Estimated size: 448
----------------------------------------------------------------------------------
Testcase 5
List Assignment
Estimated size: 1720
----------------------------------------------------------------------------------
Testcase 6
List Assignment
Estimated size: 1720
----------------------------------------------------------------------------------
Testcase 7
List Assignment
Estimated size: 116
----------------------------------------------------------------------------------
Testcase 8
Estimated size: 28
------------------------------------------

In [41]:
x = [1,2,3,4,5,6]
x *=5
print(x)

[1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6]


In [42]:
import re
x = 'y[0][i+1].append(5)'
pattern = r'(?:\[\s*[^]]+\s*\])+(?=\.\w+\s*\()'
result = re.sub(pattern, '', x)
print(result)

y.append(5)


In [43]:
x = '''y.append(float(z))'''
tree = ast.parse(x)
print(ast.dump(tree, indent=4))

Module(
    body=[
        Expr(
            value=Call(
                func=Attribute(
                    value=Name(id='y', ctx=Load()),
                    attr='append',
                    ctx=Load()),
                args=[
                    Call(
                        func=Name(id='float', ctx=Load()),
                        args=[
                            Name(id='z', ctx=Load())],
                        keywords=[])],
                keywords=[]))],
    type_ignores=[])


In [44]:
x ='''for i in x:
        y += i + 5
        names.append(y)'''
tree = ast.parse(x)
print(ast.dump(tree, indent=4))


x ='''for i in range(1000):
        x.append(10)
        x.extend(x)
        x+=5
        x[0]*=2
        y = 4 + i'''
tree = ast.parse(x)

print(ast.dump(tree, indent=4))

x ='''for i in range(len(x)):
        x.append(10)
        x.extend(x)
        x+=5
        x[0]*=2
        y = 4 + i'''
tree = ast.parse(x)

print(ast.dump(tree, indent=4))

x ='''for index, name in enumerate(names):
        x.append(y)
        a = x
        b = x'''
tree = ast.parse(x)

print(ast.dump(tree, indent=4))


x ='''for i, j in zip(a, b):
        x.extend([a,b])
       '''
tree = ast.parse(x)

print(ast.dump(tree, indent=4))

x ='''for i, j in zip(a, b):
        x.extend([a,b])
       '''
tree = ast.parse(x)

print(ast.dump(tree, indent=4))


x ='''for row in data:
        new_row = []
        for value in row:
            new_value = int(value) + 1  
            new_row.append(new_value)
       '''
tree = ast.parse(x)

print(ast.dump(tree, indent=4))

Module(
    body=[
        For(
            target=Name(id='i', ctx=Store()),
            iter=Name(id='x', ctx=Load()),
            body=[
                AugAssign(
                    target=Name(id='y', ctx=Store()),
                    op=Add(),
                    value=BinOp(
                        left=Name(id='i', ctx=Load()),
                        op=Add(),
                        right=Constant(value=5))),
                Expr(
                    value=Call(
                        func=Attribute(
                            value=Name(id='names', ctx=Load()),
                            attr='append',
                            ctx=Load()),
                        args=[
                            Name(id='y', ctx=Load())],
                        keywords=[]))],
            orelse=[])],
    type_ignores=[])
Module(
    body=[
        For(
            target=Name(id='i', ctx=Store()),
            iter=Call(
                func=Name(id='range', ctx=Load()),
          

In [45]:
transformer = InsertToAppend()
transformed_tree = transformer.visit(tree)
print(ast.dump(transformed_tree, indent=4))

Module(
    body=[
        For(
            target=Name(id='row', ctx=Store()),
            iter=Name(id='data', ctx=Load()),
            body=[
                Assign(
                    targets=[
                        Name(id='new_row', ctx=Store())],
                    value=List(elts=[], ctx=Load())),
                For(
                    target=Name(id='value', ctx=Store()),
                    iter=Name(id='row', ctx=Load()),
                    body=[
                        Assign(
                            targets=[
                                Name(id='new_value', ctx=Store())],
                            value=BinOp(
                                left=Call(
                                    func=Name(id='int', ctx=Load()),
                                    args=[
                                        Name(id='value', ctx=Load())],
                                    keywords=[]),
                                op=Add(),
                                r

In [46]:
x ='y = x.copy()'
tree = ast.parse(x)
print(ast.dump(tree, indent=4))
# parser._insertion_handler(tree)


Module(
    body=[
        Assign(
            targets=[
                Name(id='y', ctx=Store())],
            value=Call(
                func=Attribute(
                    value=Name(id='x', ctx=Load()),
                    attr='copy',
                    ctx=Load()),
                args=[],
                keywords=[]))],
    type_ignores=[])
